In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pwd

'/Users/Monika/Projects'

In [6]:
df_collisions = pd.read_csv('data/collisions_orig.csv', parse_dates=["INCDTTM"])
df_locations_xy = pd.read_csv('data/location_xy.csv')

In [8]:
# Check the starting shape
df_collisions.shape

(220436, 40)

In [9]:
# Save column list to restore column order after merge
column_list = df_collisions.columns
column_list = column_list.append(pd.Index(['fe_exists']))

In [10]:
# Drop the old X,Y and replace with the ones in location_xy.csv if they exist, otherwise, will be nan
df_collisions.drop(columns=['X','Y'],inplace=True,errors='ignore')   # get these from the location_xy.csv file
df_collisions.shape

(220436, 38)

In [11]:
df_new = pd.merge(df_collisions, df_locations_xy, on='LOCATION', how='left')

In [12]:
# Do some sanity checks
df_new.shape

(220436, 41)

In [13]:
# Restore the original order of columns
df_new = df_new[column_list]

In [14]:
df_new.drop_duplicates(inplace=True)

In [15]:
df_new['fe_exists'].value_counts()

1    216059
0      4377
Name: fe_exists, dtype: int64

In [16]:
new_keys = list(df_new['INCKEY'])
col_keys = list(df_collisions['INCKEY'])
print(f"{len(new_keys)}   {len(col_keys)}")

220436   220436


In [22]:
#Niwako - dropping exceptrsncode and expectrsndesc, deemed unnecessary
df_new.drop(['EXCEPTRSNCODE', 'EXCEPTRSNDESC'],axis=1, inplace=True)

In [27]:
#to lowercase
df_new.columns = df_new.columns.str.lower()

In [28]:
#these are % of how many nulls we have left after Jamie filling in X,Y coordinates
#consider FE flags for remaining X,Y nulls
df_new.isnull().sum()/len(df_new)

x                  0.032613
y                  0.032613
objectid           0.000000
inckey             0.000000
coldetkey          0.000000
reportno           0.000000
status             0.000000
addrtype           0.016817
intkey             0.675570
location           0.020759
severitycode       0.000005
severitydesc       0.000000
collisiontype      0.118638
personcount        0.000000
pedcount           0.000000
pedcylcount        0.000000
vehcount           0.000000
injuries           0.000000
seriousinjuries    0.000000
fatalities         0.000000
incdate            0.000000
incdttm            0.000000
junctiontype       0.054147
sdot_colcode       0.000005
sdot_coldesc       0.000005
inattentionind     0.863058
underinfl          0.118547
weather            0.119490
roadcond           0.119128
lightcond          0.119894
pedrownotgrnt      0.976501
sdotcolnum         0.422939
speeding           0.955198
st_colcode         0.042702
st_coldesc         0.118638
seglanekey         0

In [31]:
print(df_new['x'].isnull().sum())
print(df_new['y'].isnull().sum())

7189
7189


In [26]:
#exporting to a csv, this will overwrite when more people fill in their nulls
df_new.to_csv('collisions_clean.csv')